# 5章 顧客の退会を予測する１０本ノック

引き続き、スポーツジムの会員データを使って顧客の行動を分析していきます。  
３章では顧客の全体像を把握し、4章では数ヶ月利用している顧客の来月の利用回数の予測を行いました。   
ここでは、教師あり学習の分類を用いて、顧客の退会予測を取り扱います。

### ノック41：データを読み込んで利用データを整形しよう

In [114]:
import pandas as pd
customer = pd.read_csv('customer_join.csv')
print(customer)
uselog_months = pd.read_csv('use_log_months.csv')
print(uselog_months)

customer_id   name class  ... routine_flg   calc_date membership_period
0       OA832399   XXXX   C01  ...           1  2019-04-30                47
1       PL270116  XXXXX   C01  ...           1  2019-04-30                47
2       OA974876  XXXXX   C01  ...           1  2019-04-30                47
3       HD024127  XXXXX   C01  ...           1  2019-04-30                47
4       HD661448  XXXXX   C03  ...           1  2019-04-30                47
...          ...    ...   ...  ...         ...         ...               ...
4187    HD676663   XXXX   C01  ...           0  2019-04-30                 1
4188    HD246549  XXXXX   C01  ...           0  2019-04-30                 1
4189    GD037007  XXXXX   C03  ...           0  2019-04-30                 1
4190    OA953150  XXXXX   C01  ...           0  2019-04-30                 1
4191    IK692635  XXXXX   C02  ...           0  2019-04-30                 1

[4192 rows x 18 columns]
           年月 customer_id  count
0      201804    AS002

In [115]:
year_months = list(uselog_months['年月'].unique())
print(year_months)
uselog = pd.DataFrame()

for i in range(1, len(year_months)):
    tmp = uselog_months.loc[uselog_months['年月'] == year_months[i]]
    tmp.rename(columns={'count': 'count_0'}, inplace=True)
    tmp_before = uselog_months.loc[uselog_months['年月'] == year_months[i-1]]
    del tmp_before['年月']
    tmp_before.rename(columns={'count': 'count_1'}, inplace=True)
    tmp = pd.merge(tmp, tmp_before, on='customer_id', how='left')
    uselog = pd.concat([uselog, tmp], ignore_index=True)

uselog



[201804, 201805, 201806, 201807, 201808, 201809, 201810, 201811, 201812, 201901, 201902, 201903]


,年月,customer_id,count_0,count_1
0,201805,AS002855,5,4.0
1,201805,AS009373,4,3.0
2,201805,AS015233,7,NaN
3,201805,AS015315,3,6.0
4,201805,AS015739,5,7.0
...,...,...,...,...
33846,201903,TS995853,8,11.0
33847,201903,TS998593,8,7.0
33848,201903,TS999079,3,2.0
33849,201903,TS999231,6,6.0


### ノック42：退会前月の退会顧客データを作成しよう

In [116]:
from dateutil.relativedelta import relativedelta
exit_customer = customer.loc[customer['is_deleted'] == 1]
exit_customer['exit_date'] = None
exit_customer['end_date'] = pd.to_datetime(exit_customer['end_date'])
for i in range(len(exit_customer)):
    exit_customer['exit_date'].iloc[i] = exit_customer['end_date'].iloc[i] - relativedelta(months=1)
exit_customer['年月'] = exit_customer['exit_date'].dt.strftime('%Y%m')
uselog['年月'] = uselog['年月'].astype(str)
exit_uselog = pd.merge(uselog, exit_customer, on=['customer_id', '年月'], how='left')
print(len(uselog))
exit_uselog

33851


,年月,customer_id,count_0,count_1,name,class,gender,start_date,end_date,campaign_id,is_deleted,class_name,price,campaign_name,mean,median,max,min,routine_flg,calc_date,membership_period,exit_date
0,201805,AS002855,5,4.0,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,201805,AS009373,4,3.0,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,201805,AS015233,7,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,201805,AS015315,3,6.0,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,201805,AS015739,5,7.0,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33846,201903,TS995853,8,11.0,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
33847,201903,TS998593,8,7.0,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
33848,201903,TS999079,3,2.0,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
33849,201903,TS999231,6,6.0,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [117]:
exit_uselog = exit_uselog.dropna(subset=['name'])
exit_uselog

,年月,customer_id,count_0,count_1,name,class,gender,start_date,end_date,campaign_id,is_deleted,class_name,price,campaign_name,mean,median,max,min,routine_flg,calc_date,membership_period,exit_date
19,201805,AS055680,3,3.0,XXXXX,C01,M,2018-03-01,2018-06-30,CA1,1.0,オールタイム,10500.0,通常,3.000000,3.0,3.0,3.0,0.0,2018-06-30,3.0,2018-05-30 00:00:00
57,201805,AS169823,2,3.0,XX,C01,M,2017-11-01,2018-06-30,CA1,1.0,オールタイム,10500.0,通常,3.000000,3.0,4.0,2.0,1.0,2018-06-30,7.0,2018-05-30 00:00:00
110,201805,AS305860,5,3.0,XXXX,C01,M,2017-06-01,2018-06-30,CA1,1.0,オールタイム,10500.0,通常,3.333333,3.0,5.0,2.0,0.0,2018-06-30,12.0,2018-05-30 00:00:00
128,201805,AS363699,5,3.0,XXXXX,C01,M,2018-02-01,2018-06-30,CA1,1.0,オールタイム,10500.0,通常,3.333333,3.0,5.0,2.0,0.0,2018-06-30,4.0,2018-05-30 00:00:00
147,201805,AS417696,1,4.0,XX,C03,F,2017-09-01,2018-06-30,CA1,1.0,ナイト,6000.0,通常,2.000000,1.0,4.0,1.0,0.0,2018-06-30,9.0,2018-05-30 00:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30758,201902,TS645212,4,2.0,XXXX,C03,F,2018-03-01,2019-03-31,CA1,1.0,ナイト,6000.0,通常,4.500000,4.5,7.0,1.0,0.0,2019-03-31,12.0,2019-02-28 00:00:00
30787,201902,TS741703,5,6.0,XXXX,C03,M,2018-12-08,2019-03-31,CA3,1.0,ナイト,6000.0,入会費無料,6.250000,6.0,8.0,5.0,0.0,2019-03-31,3.0,2019-02-28 00:00:00
30827,201902,TS859258,1,3.0,XXXXX,C02,F,2018-12-07,2019-03-31,CA3,1.0,デイタイム,7500.0,入会費無料,2.500000,2.0,5.0,1.0,0.0,2019-03-31,3.0,2019-02-28 00:00:00
30842,201902,TS886985,5,3.0,XXX,C02,F,2018-03-01,2019-03-31,CA1,1.0,デイタイム,7500.0,通常,4.250000,4.0,7.0,2.0,1.0,2019-03-31,12.0,2019-02-28 00:00:00


### ノック43：継続顧客のデータを作成しよう

In [118]:
conti_cusomer = customer.loc[customer['is_deleted'] == 0]
conti_uselog = pd.merge(uselog, conti_cusomer, on=['customer_id'], how='left')
print(len(conti_uselog))
conti_uselog = conti_uselog.dropna(subset=['name'])
print(len(conti_uselog))

33851
27422


In [119]:
conti_uselog = conti_uselog.sample(frac=1).reset_index(drop=True)
conti_uselog = conti_uselog.drop_duplicates(subset='customer_id')
conti_uselog

,年月,customer_id,count_0,count_1,name,class,gender,start_date,end_date,campaign_id,is_deleted,class_name,price,campaign_name,mean,median,max,min,routine_flg,calc_date,membership_period
0,201901,TS260662,5,5.0,XXXXX,C02,M,2015-12-01,NaN,CA1,0.0,デイタイム,7500.0,通常,4.583333,5.0,6.0,3.0,1.0,2019-04-30,40.0
1,201902,TS020727,8,6.0,XX,C01,M,2017-05-01,NaN,CA1,0.0,オールタイム,10500.0,通常,6.083333,6.0,8.0,3.0,1.0,2019-04-30,23.0
2,201901,PL842727,7,9.0,XXX,C02,M,2018-12-06,NaN,CA3,0.0,デイタイム,7500.0,入会費無料,8.250000,8.0,10.0,7.0,1.0,2019-04-30,4.0
3,201811,GD975752,5,6.0,XXXX,C02,F,2016-08-01,NaN,CA1,0.0,デイタイム,7500.0,通常,5.250000,5.0,7.0,2.0,1.0,2019-04-30,32.0
4,201811,OA449475,5,5.0,XXXXXX,C01,M,2017-09-01,NaN,CA1,0.0,オールタイム,10500.0,通常,5.833333,5.5,9.0,1.0,1.0,2019-04-30,19.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26499,201903,AS739640,10,NaN,XXXXXX,C03,M,2019-03-13,NaN,CA1,0.0,ナイト,6000.0,通常,10.000000,10.0,10.0,10.0,0.0,2019-04-30,1.0
26511,201903,PL425572,7,NaN,XXXX,C01,F,2019-03-12,NaN,CA1,0.0,オールタイム,10500.0,通常,7.000000,7.0,7.0,7.0,0.0,2019-04-30,1.0
26724,201903,OA643695,11,NaN,XXXXX,C02,F,2019-03-12,NaN,CA1,0.0,デイタイム,7500.0,通常,11.000000,11.0,11.0,11.0,0.0,2019-04-30,1.0
26782,201903,TS926301,8,NaN,XXXXXX,C02,F,2019-03-08,NaN,CA1,0.0,デイタイム,7500.0,通常,8.000000,8.0,8.0,8.0,1.0,2019-04-30,1.0


In [120]:
predict_data = pd.concat([conti_uselog, exit_uselog], ignore_index=True)
predict_data

,年月,customer_id,count_0,count_1,name,class,gender,start_date,end_date,campaign_id,is_deleted,class_name,price,campaign_name,mean,median,max,min,routine_flg,calc_date,membership_period,exit_date
0,201901,TS260662,5,5.0,XXXXX,C02,M,2015-12-01,NaN,CA1,0.0,デイタイム,7500.0,通常,4.583333,5.0,6.0,3.0,1.0,2019-04-30,40.0,NaN
1,201902,TS020727,8,6.0,XX,C01,M,2017-05-01,NaN,CA1,0.0,オールタイム,10500.0,通常,6.083333,6.0,8.0,3.0,1.0,2019-04-30,23.0,NaN
2,201901,PL842727,7,9.0,XXX,C02,M,2018-12-06,NaN,CA3,0.0,デイタイム,7500.0,入会費無料,8.250000,8.0,10.0,7.0,1.0,2019-04-30,4.0,NaN
3,201811,GD975752,5,6.0,XXXX,C02,F,2016-08-01,NaN,CA1,0.0,デイタイム,7500.0,通常,5.250000,5.0,7.0,2.0,1.0,2019-04-30,32.0,NaN
4,201811,OA449475,5,5.0,XXXXXX,C01,M,2017-09-01,NaN,CA1,0.0,オールタイム,10500.0,通常,5.833333,5.5,9.0,1.0,1.0,2019-04-30,19.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3941,201902,TS645212,4,2.0,XXXX,C03,F,2018-03-01,2019-03-31 00:00:00,CA1,1.0,ナイト,6000.0,通常,4.500000,4.5,7.0,1.0,0.0,2019-03-31,12.0,2019-02-28 00:00:00
3942,201902,TS741703,5,6.0,XXXX,C03,M,2018-12-08,2019-03-31 00:00:00,CA3,1.0,ナイト,6000.0,入会費無料,6.250000,6.0,8.0,5.0,0.0,2019-03-31,3.0,2019-02-28 00:00:00
3943,201902,TS859258,1,3.0,XXXXX,C02,F,2018-12-07,2019-03-31 00:00:00,CA3,1.0,デイタイム,7500.0,入会費無料,2.500000,2.0,5.0,1.0,0.0,2019-03-31,3.0,2019-02-28 00:00:00
3944,201902,TS886985,5,3.0,XXX,C02,F,2018-03-01,2019-03-31 00:00:00,CA1,1.0,デイタイム,7500.0,通常,4.250000,4.0,7.0,2.0,1.0,2019-03-31,12.0,2019-02-28 00:00:00


### ノック44：予測する月の在籍期間を作成しよう

In [121]:

# predict_data['end_date'].unique()
# print(predict_data.dtypes)

predict_data['period'] = 0
predict_data['now_date'] = pd.to_datetime(predict_data['年月'], format='%Y%m')
predict_data['start_date'] = pd.to_datetime(predict_data['start_date'])

from dateutil.relativedelta import relativedelta

for i in range(len(predict_data)):
    d = relativedelta(predict_data['now_date'][i], predict_data['start_date'][i])
    predict_data['period'][i] = int(d.years * 12 + d.months)

predict_data



,年月,customer_id,count_0,count_1,name,class,gender,start_date,end_date,campaign_id,is_deleted,class_name,price,campaign_name,mean,median,max,min,routine_flg,calc_date,membership_period,exit_date,period,now_date
0,201901,TS260662,5,5.0,XXXXX,C02,M,2015-12-01,NaN,CA1,0.0,デイタイム,7500.0,通常,4.583333,5.0,6.0,3.0,1.0,2019-04-30,40.0,NaN,37,2019-01-01
1,201902,TS020727,8,6.0,XX,C01,M,2017-05-01,NaN,CA1,0.0,オールタイム,10500.0,通常,6.083333,6.0,8.0,3.0,1.0,2019-04-30,23.0,NaN,21,2019-02-01
2,201901,PL842727,7,9.0,XXX,C02,M,2018-12-06,NaN,CA3,0.0,デイタイム,7500.0,入会費無料,8.250000,8.0,10.0,7.0,1.0,2019-04-30,4.0,NaN,0,2019-01-01
3,201811,GD975752,5,6.0,XXXX,C02,F,2016-08-01,NaN,CA1,0.0,デイタイム,7500.0,通常,5.250000,5.0,7.0,2.0,1.0,2019-04-30,32.0,NaN,27,2018-11-01
4,201811,OA449475,5,5.0,XXXXXX,C01,M,2017-09-01,NaN,CA1,0.0,オールタイム,10500.0,通常,5.833333,5.5,9.0,1.0,1.0,2019-04-30,19.0,NaN,14,2018-11-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3941,201902,TS645212,4,2.0,XXXX,C03,F,2018-03-01,2019-03-31 00:00:00,CA1,1.0,ナイト,6000.0,通常,4.500000,4.5,7.0,1.0,0.0,2019-03-31,12.0,2019-02-28 00:00:00,11,2019-02-01
3942,201902,TS741703,5,6.0,XXXX,C03,M,2018-12-08,2019-03-31 00:00:00,CA3,1.0,ナイト,6000.0,入会費無料,6.250000,6.0,8.0,5.0,0.0,2019-03-31,3.0,2019-02-28 00:00:00,1,2019-02-01
3943,201902,TS859258,1,3.0,XXXXX,C02,F,2018-12-07,2019-03-31 00:00:00,CA3,1.0,デイタイム,7500.0,入会費無料,2.500000,2.0,5.0,1.0,0.0,2019-03-31,3.0,2019-02-28 00:00:00,1,2019-02-01
3944,201902,TS886985,5,3.0,XXX,C02,F,2018-03-01,2019-03-31 00:00:00,CA1,1.0,デイタイム,7500.0,通常,4.250000,4.0,7.0,2.0,1.0,2019-03-31,12.0,2019-02-28 00:00:00,11,2019-02-01


### ノック45：欠損値を除去しよう

In [122]:
predict_data.isna().sum()

年月                      0
customer_id             0
count_0                 0
count_1               266
name                    0
class                   0
gender                  0
start_date              0
end_date             2842
campaign_id             0
is_deleted              0
class_name              0
price                   0
campaign_name           0
mean                    0
median                  0
max                     0
min                     0
routine_flg             0
calc_date               0
membership_period       0
exit_date            2842
period                  0
now_date                0
dtype: int64

In [123]:
predict_data = predict_data.dropna(subset=['count_1'])
predict_data.isna().sum()

年月                      0
customer_id             0
count_0                 0
count_1                 0
name                    0
class                   0
gender                  0
start_date              0
end_date             2628
campaign_id             0
is_deleted              0
class_name              0
price                   0
campaign_name           0
mean                    0
median                  0
max                     0
min                     0
routine_flg             0
calc_date               0
membership_period       0
exit_date            2628
period                  0
now_date                0
dtype: int64

### ノック46：文字列型の変数を処理できるように整形しよう

In [124]:
target_col = ['campaign_name', 'class_name', 'gender', 'count_1', 'routine_flg', 'period', 'is_deleted']
predict_data = predict_data[target_col]
predict_data

,campaign_name,class_name,gender,count_1,routine_flg,period,is_deleted
0,通常,デイタイム,M,5.0,1.0,37,0.0
1,通常,オールタイム,M,6.0,1.0,21,0.0
2,入会費無料,デイタイム,M,9.0,1.0,0,0.0
3,通常,デイタイム,F,6.0,1.0,27,0.0
4,通常,オールタイム,M,5.0,1.0,14,0.0
...,...,...,...,...,...,...,...
3941,通常,ナイト,F,2.0,0.0,11,1.0
3942,入会費無料,ナイト,M,6.0,0.0,1,1.0
3943,入会費無料,デイタイム,F,3.0,0.0,1,1.0
3944,通常,デイタイム,F,3.0,1.0,11,1.0


In [125]:
predict_data = pd.get_dummies(predict_data)
predict_data

,count_1,routine_flg,period,is_deleted,campaign_name_入会費半額,campaign_name_入会費無料,campaign_name_通常,class_name_オールタイム,class_name_デイタイム,class_name_ナイト,gender_F,gender_M
0,5.0,1.0,37,0.0,0,0,1,0,1,0,0,1
1,6.0,1.0,21,0.0,0,0,1,1,0,0,0,1
2,9.0,1.0,0,0.0,0,1,0,0,1,0,0,1
3,6.0,1.0,27,0.0,0,0,1,0,1,0,1,0
4,5.0,1.0,14,0.0,0,0,1,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...
3941,2.0,0.0,11,1.0,0,0,1,0,0,1,1,0
3942,6.0,0.0,1,1.0,0,1,0,0,0,1,0,1
3943,3.0,0.0,1,1.0,0,1,0,0,1,0,1,0
3944,3.0,1.0,11,1.0,0,0,1,0,1,0,1,0


In [126]:
del predict_data['campaign_name_通常']
del predict_data['class_name_ナイト']
del predict_data['gender_M']

### ノック47：決定木を用いて退会予測モデルを作成してみよう

In [127]:
from sklearn.tree import DecisionTreeClassifier
import sklearn.model_selection

exit = predict_data.loc[predict_data['is_deleted'] == 1]
conti = predict_data.loc[predict_data['is_deleted'] == 0].sample(len(exit))

X = pd.concat([exit, conti], ignore_index=True)
y = X['is_deleted']
del X['is_deleted']

X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X, y)

model = DecisionTreeClassifier()
model.fit(X_train, y_train)
y_test_pred = model.predict(X_test)
y_test_pred


array([0., 1., 0., 0., 0., 1., 0., 1., 0., 1., 1., 1., 1., 1., 0., 0., 0.,
       1., 0., 1., 1., 1., 0., 0., 1., 1., 1., 0., 0., 1., 0., 1., 1., 1.,
       1., 1., 1., 1., 0., 0., 1., 0., 1., 1., 0., 1., 1., 0., 0., 0., 0.,
       1., 0., 1., 1., 1., 0., 1., 1., 1., 1., 1., 0., 0., 1., 0., 1., 0.,
       0., 0., 1., 0., 1., 1., 0., 1., 1., 0., 1., 1., 0., 0., 1., 1., 0.,
       0., 0., 0., 0., 0., 1., 1., 0., 1., 0., 0., 0., 0., 0., 0., 0., 1.,
       1., 1., 1., 0., 1., 0., 1., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0.,
       0., 1., 0., 1., 1., 1., 0., 0., 1., 0., 0., 1., 1., 1., 1., 0., 1.,
       0., 1., 1., 0., 0., 0., 0., 0., 1., 1., 1., 0., 0., 1., 1., 0., 0.,
       0., 1., 1., 1., 0., 1., 1., 0., 0., 0., 0., 1., 1., 0., 0., 1., 0.,
       1., 0., 1., 1., 1., 1., 0., 1., 0., 1., 0., 0., 0., 0., 0., 1., 0.,
       0., 1., 1., 1., 1., 0., 1., 0., 0., 1., 1., 1., 1., 0., 1., 0., 1.,
       1., 1., 0., 0., 1., 0., 1., 1., 0., 0., 0., 1., 1., 0., 0., 1., 0.,
       1., 0., 0., 1., 1.

In [128]:
results_test = pd.DataFrame({'y_test': y_test, 'y_pred': y_test_pred})
results_test

,y_test,y_pred
1305,0.0,0.0
826,1.0,1.0
1158,0.0,0.0
798,1.0,0.0
1952,0.0,0.0
...,...,...
100,1.0,1.0
786,1.0,1.0
140,1.0,1.0
914,1.0,1.0


### ノック48：予測モデルの評価を行ない、モデルのチューニングをしてみよう

In [129]:
correct = len(results_test.loc[results_test['y_test'] == results_test['y_pred']])
data_count = len(results_test)
score_test = correct / data_count
score_test


0.8802281368821293

In [130]:
print(model.score(X_test, y_test))
print(model.score(X_train, y_train))

0.8802281368821293
0.9816223067173637


In [131]:
X = pd.concat([exit, conti], ignore_index=True)
y = X['is_deleted']
del X['is_deleted']

X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X, y)

model = DecisionTreeClassifier(random_state=0, max_depth=5)
model.fit(X_train, y_train)
print(model.score(X_test, y_test))
print(model.score(X_train, y_train))

0.9163498098859315
0.9207858048162231


### ノック49：モデルに寄与している変数を確認しよう

In [132]:
importance = pd.DataFrame({'feature_names': X.columns, 'coefficient': model.feature_importances_})
importance

,feature_names,coefficient
0,count_1,0.338621
1,routine_flg,0.119191
2,period,0.536829
3,campaign_name_入会費半額,0.000000
4,campaign_name_入会費無料,0.005359
5,class_name_オールタイム,0.000000
6,class_name_デイタイム,0.000000
7,gender_F,0.000000


### ノック50：顧客の退会を予測しよう

In [133]:
count_1 = 3
routine_flg = 1
period = 10
campaign_name = '入会費無料'
class_name = 'オールタイム'
gender = 'M'

In [136]:
if campaign_name == '入会費半額':
    campaign_name_list = [1, 0]
elif campaign_name == '入会費無料':
    campaign_name_list = [0, 1]
else:
    campaign_name_list = [0, 0]

if class_name == 'オールタイム':
    class_name_list = [1, 0]
elif class_name == 'デイタイム':
    class_name_list = [0, 1]
else:
    class_name_list = [0, 0]

if gender == 'F':
    gender_list = [1]
else:
    gender_list = [0]

input_data = [count_1, routine_flg, period]
input_data.extend(campaign_name_list)
input_data.extend(class_name_list)
input_data.extend(gender_list)

input_data

[3, 1, 10, 0, 1, 1, 0, 0]

In [135]:
print(model.predict([input_data]))
print(model.predict_proba([input_data]))

[1.]


NameError: name 'input_daat' is not defined